# App for Lottery Addiction

In this project we will be building an app which enables users to understand the probability of winning the [Lotto 6/49](https://en.wikipedia.org/wiki/Lotto_6/49), or to understand the likelihood of other outcomes occuring such as: 

- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

We will use a [Lotto 6/49 dataset](https://www.kaggle.com/datascienceai/lottery-dataset) which contains 3,665 drawings between 1982 and 2018.

The focus of the app is to raise awareness of probability amongst gamblers, with the aim that by providing this knowledge it encourages people to gamble responsibily and reduce risk of gambling addiction.

## Core Functions

We will start by building two core functions which we will use throughout the project, one for factorials and another for combinations.

In [2]:
def factorial(n):
    final_product = 1
    for i in range(n, 0, -1):
        final_product *= i
    return final_product

def combinations(n, k):
    numerator = factorial(n)
    denominator = factorial(k) * factorial(n-k)
    return numerator / denominator                             

## One Ticket Probability

Now we will build a function that calculates the probability of winning the jackpot. To enable this, the application will need to:

- Allow users to input six different numbers from 1 to 49
- Store these 6 input numbers as a single python list, which becomes the input variable for our function
- Print the probability in a user-friendly way so it can be clearly understood

In [9]:
def one_ticket_probability(user_numbers):
    outcomes = combinations(49, 6)
    p_one_ticket = 1/outcomes
    pct_format = p_one_ticket*100
    print('''Your chances of winning the jackpot with numbers {} is {:.7f}%.
    In other words, you have a 1 in {:,} chance of winning.'''.format(user_numbers, pct_format, int(outcomes)))

In [10]:
test_one = one_ticket_probability([1,2,3,4,5,6])
test_one

Your chances of winning the jackpot with numbers [1, 2, 3, 4, 5, 6] is 0.0000072%.
    In other words, you have a 1 in 13,983,816 chance of winning.


In [11]:
test_two = one_ticket_probability([8,13,20,23,27,38])
test_two

Your chances of winning the jackpot with numbers [8, 13, 20, 23, 27, 38] is 0.0000072%.
    In other words, you have a 1 in 13,983,816 chance of winning.


## Historical Data Check

For our first version of the app we would like users to be able to input numbers and compare their ticket to historic draws to see if they would have won by now. 

We will start by reading in the dataset and familiarising ourselves with its content. 

In [15]:
import pandas as pd
lottery = pd.read_csv('649.csv')
print(lottery.shape)
print(lottery.head(1), lottery.tail(3))

(3665, 11)
   PRODUCT  DRAW NUMBER  SEQUENCE NUMBER  DRAW DATE  NUMBER DRAWN 1  \
0      649            1                0  6/12/1982               3   

   NUMBER DRAWN 2  NUMBER DRAWN 3  NUMBER DRAWN 4  NUMBER DRAWN 5  \
0              11              12              14              41   

   NUMBER DRAWN 6  BONUS NUMBER  
0              43            13         PRODUCT  DRAW NUMBER  SEQUENCE NUMBER  DRAW DATE  NUMBER DRAWN 1  \
3662      649         3589                0  6/13/2018               6   
3663      649         3590                0  6/16/2018               2   
3664      649         3591                0  6/20/2018              14   

      NUMBER DRAWN 2  NUMBER DRAWN 3  NUMBER DRAWN 4  NUMBER DRAWN 5  \
3662              22              24              31              32   
3663              15              21              31              38   
3664              24              31              35              37   

      NUMBER DRAWN 6  BONUS NUMBER  
3662            

## Function for Historical Data Check

Now we will create a function enabling users to compare their ticket against historial lottery data to determine whether they would have won by now. 

The function will perform similarly to 'one_ticket_probability' as coded earlier, with the addition of returing the number of times the combination selected occurred in the data set.

In [17]:
def extract_numbers(row):
    row = row[4:10]
    row = set(row.values)
    return row

winning_numbers = lottery.apply(extract_numbers, axis=1)
winning_numbers.head()

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
3     {3, 9, 10, 43, 13, 20}
4    {34, 5, 14, 47, 21, 31}
dtype: object

In [30]:
def check_historical_occurence(user_numbers, winning_numbers):
    user_numbers = set(user_numbers)
    check_occurence = user_numbers == winning_numbers
    occurences = check_occurence.sum()
    if occurences == 0:
        print('''The combination {} has never won the lottery previously. This doesn't mean it is more likely to occur in the future.  Your chances of winning the jackpot with these numbers {} in the next draw are 0.0000072%. 
In other words, you have a 1 in 13,983,816 chance of winning.'''.format(user_numbers, user_numbers))
    else:
        print('''This combination {} has won {} times between 1982 and 2018. Your chances of winning the jackpot with these numbers {} in the next draw are 0.0000072%.
In other words, you have a 1 in 13,983,816 chance of winning.'''.format(user_numbers, occurences, user_numbers))

In [31]:
hist_test_1 = check_historical_occurence([3,5,8,13,20,23], winning_numbers)
hist_test_1

The combination {3, 5, 8, 13, 20, 23} has never won the lottery previously. This doesn't mean it is more likely to occur in the future.  Your chances of winning the jackpot with these numbers {3, 5, 8, 13, 20, 23} in the next draw are 0.0000072%. 
In other words, you have a 1 in 13,983,816 chance of winning.


In [32]:
hist_test_2 = check_historical_occurence([1, 6, 39, 23, 24, 27], winning_numbers)
hist_test_2

This combination {1, 6, 39, 23, 24, 27} has won 1 times between 1982 and 2018. Your chances of winning the jackpot with these numbers {1, 6, 39, 23, 24, 27} in the next draw are 0.0000072%.
In other words, you have a 1 in 13,983,816 chance of winning.


## Multi-ticket Probability
Given people are likely to increase their gambling volume over time, it is important that we also create a function for checking probability of winning the jackpot based on purchasing multiple tickets.

For this function:
- The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
- We will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
- It will print information about the probability of winning the big prize depending on the number of different tickets played.

In [55]:
def multi_ticket_probability(num_tickets):
    outcomes = combinations(49, 6)
    probability = num_tickets/outcomes
    prob_pct = probability * 100
    
    if num_tickets == 1:
        print('''Your chances of winning the jackpot with one ticket are {:.6f}%.
In other words, you have a 1 in {:,} chance to win'''.format(prob_pct, int(outcomes)))
    else:
        outcomes_v2 = round(outcomes/num_tickets)
        print('''Your chances of winning the jackpot with {} tickets is {:.6f}%.
In other words, you have a 1 in {:,} chance of winning.'''.format(num_tickets, prob_pct, outcomes_v2))

In [56]:
test_inputs = [1, 10, 100, 10000, 1000000, 6991908, 13983816]

for test_input in test_inputs:
    multi_ticket_probability(test_input)
    print('------------------')

Your chances of winning the jackpot with one ticket are 0.000007%.
In other words, you have a 1 in 13,983,816 chance to win
------------------
Your chances of winning the jackpot with 10 tickets is 0.000072%.
In other words, you have a 1 in 1,398,382 chance of winning.
------------------
Your chances of winning the jackpot with 100 tickets is 0.000715%.
In other words, you have a 1 in 139,838 chance of winning.
------------------
Your chances of winning the jackpot with 10000 tickets is 0.071511%.
In other words, you have a 1 in 1,398 chance of winning.
------------------
Your chances of winning the jackpot with 1000000 tickets is 7.151124%.
In other words, you have a 1 in 14 chance of winning.
------------------
Your chances of winning the jackpot with 6991908 tickets is 50.000000%.
In other words, you have a 1 in 2 chance of winning.
------------------
Your chances of winning the jackpot with 13983816 tickets is 100.000000%.
In other words, you have a 1 in 1 chance of winning.
------

## Less Winning Numbers

We're going to create one more function, which will allow users to calculate the probabilities for returing two, three, four or five winning numbers. This is useful as there are smaller prizes still availabe for matching some, rather than all, numbers. 

For this function, inside the app the user inputs:
- Six different numbers from 1 to 49; and
- An integer between 2 and 5 that represents the number of winning numbers expected
Our function prints information about the probability of having the inputted number of winning numbers.

In [64]:
def probability_less_6(n_winning_numbers):
    combinations_ticket = combinations(6, n_winning_numbers)
    outcomes_remaining = combinations(43, 6-n_winning_numbers)
    combinations_total = combinations(49, 6)
    s_outcomes = combinations_ticket * outcomes_remaining
    probability = s_outcomes/combinations_total
    probability_pct = probability * 100
    outcomes_v2 = round(combinations_total/s_outcomes)
    print('''Your chances of winning with {} numbers is {:.7f}%.
    In other words, you have a 1 in {:,} chance of winning.'''.format(n_winning_numbers, probability_pct, int(outcomes_v2)))

In [68]:
tst_2 = probability_less_6(2)
print(tst_2)

Your chances of winning with 2 numbers is 13.2378029%.
    In other words, you have a 1 in 8 chance of winning.
None
